In [7]:
import pandas as pd
# from dowhy import gcm
import numpy as np
import matplotlib.pyplot as plt
import gc

import os

In [8]:
#import exp_filtered
data = pd.read_csv('./Extraction/experiments_filtered.csv',header=0, index_col=0)
experiments = pd.DataFrame(data)

experiments.tail(10)


,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
288,745,2024-01-19 20:52:52.8522174,2024-01-19 21:22:52.8564323,general,random27,8,1,23,4,8,2,0.25,3.0,1705697572,1705699372
289,746,2024-01-19 21:26:27.3234593,2024-01-19 21:56:27.3265222,general,random27,8,1,23,4,8,2,0.25,3.0,1705699587,1705701387
290,747,2024-01-19 22:00:01.6380175,2024-01-19 22:30:01.6457071,general,random28,20,18,23,8,4,16,0.50,1.0,1705701601,1705703401
294,751,2024-01-20 00:14:24.4097982,2024-01-20 00:44:24.4126883,general,random29,18,20,8,4,6,4,0.25,3.0,1705709664,1705711464
295,752,2024-01-20 00:47:58.7269655,2024-01-20 01:17:58.7312773,general,random29,18,20,8,4,6,4,0.25,3.0,1705711678,1705713478
296,753,2024-01-20 01:21:32.6299318,2024-01-20 01:51:32.6406703,general,random30,15,4,20,8,2,8,1.00,1.0,1705713692,1705715492
299,758,2024-01-20 04:09:25.0644731,2024-01-20 04:39:25.0711445,general,random31,8,6,18,4,4,4,0.25,0.5,1705723765,1705725565
301,760,2024-01-20 05:16:34.1638368,2024-01-20 05:46:34.1666149,general,random32,3,2,5,2,2,2,0.50,1.0,1705727794,1705729594
302,761,2024-01-20 05:50:08.2461471,2024-01-20 06:20:08.2541699,general,random32,3,2,5,2,2,2,0.50,1.0,1705729808,1705731608
303,762,2024-01-20 06:23:42.6007382,2024-01-20 06:53:42.6043583,general,random33,19,8,23,4,8,2,0.25,3.0,1705731822,1705733622


In [9]:
messages = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)
    query_mess = query_mess.drop(columns=["start", "end"])

    messages = pd.concat([query_mess, messages])
    del query_mess
    gc.collect()

    if os.path.exists('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/ihave_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_ihave = pd.DataFrame(data)
        query_ihave["type"] = '17.0'
        query_ihave = query_ihave.drop(columns=["_measurement"])
        
        messages = pd.concat([query_ihave, messages])
        del query_ihave
        gc.collect()
    
    if os.path.exists('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv'):
        data = pd.read_csv('../../datasets/partialTables/iwant_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
        query_iwant = pd.DataFrame(data)
        query_iwant["type"] = '16.0'
        query_iwant = query_iwant.drop(columns=["_measurement"])
    
        messages = pd.concat([query_iwant, messages])
        del query_iwant
        gc.collect()
    

messages.head(10)

,_time,identifier,experiment,type
0,2024-01-20 06:31:00.348127+00:00,303,762,16.0
1,2024-01-20 06:46:45.354625+00:00,303,762,16.0
2,2024-01-20 06:46:45.355063+00:00,303,762,16.0
3,2024-01-20 06:46:57.349052+00:00,303,762,16.0
0,2024-01-20 06:23:43.925455+00:00,303,762,17.0
1,2024-01-20 06:23:43.967483+00:00,303,762,17.0
2,2024-01-20 06:23:43.999768+00:00,303,762,17.0
3,2024-01-20 06:23:43.999801+00:00,303,762,17.0
4,2024-01-20 06:23:44.000139+00:00,303,762,17.0
5,2024-01-20 06:23:44.005508+00:00,303,762,17.0


In [10]:
# Transpose the table -> type and count
df = messages.drop(columns=['identifier'])
df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],df['type'],pd.Grouper(key="_time", freq='5s')])['index'].count().reset_index()


pivoted = pd.pivot_table(by_time,index=["experiment", pd.Grouper(freq="5s", key="_time")], columns="type", values="index", margins=False).fillna(0)
pivoted = pivoted.reset_index().rename_axis(None, axis=1)

pivoted.head(100)

/tmp/ipykernel_3299/847033619.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,2.0,3.0,11.0,12.0,16.0,17.0
0,390,2023-10-04 18:52:00+00:00,3748.0,520.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,4936.0,673.0,0.0,0.0,0.0,698.0
2,390,2023-10-04 18:52:10+00:00,8457.0,1156.0,0.0,0.0,0.0,689.0
3,390,2023-10-04 18:52:15+00:00,7681.0,1032.0,0.0,0.0,0.0,343.0
4,390,2023-10-04 18:52:20+00:00,5874.0,796.0,0.0,0.0,0.0,695.0
...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,4378.0,654.0,0.0,0.0,0.0,393.0
96,390,2023-10-04 19:00:00+00:00,2748.0,408.0,0.0,0.0,0.0,195.0
97,390,2023-10-04 19:00:05+00:00,4453.0,669.0,0.0,0.0,0.0,385.0
98,390,2023-10-04 19:00:10+00:00,4292.0,650.0,0.0,0.0,0.0,357.0


In [11]:
#Rename
eventsTS = pivoted.rename(columns={2.0:'messageDuplicated', 3.0: 'messageReceived', 11.0: 'graft', 12.0: 'prune', '16.0': 'iwant', '17.0':'ihave'})

eventsTS.to_csv('./events5s.csv')
eventsTS.head(500)

,experiment,_time,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,3748.0,520.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,4936.0,673.0,0.0,0.0,0.0,698.0
2,390,2023-10-04 18:52:10+00:00,8457.0,1156.0,0.0,0.0,0.0,689.0
3,390,2023-10-04 18:52:15+00:00,7681.0,1032.0,0.0,0.0,0.0,343.0
4,390,2023-10-04 18:52:20+00:00,5874.0,796.0,0.0,0.0,0.0,695.0
...,...,...,...,...,...,...,...,...
495,391,2023-10-04 19:43:55+00:00,10820.0,1634.0,0.0,0.0,1.0,814.0
496,391,2023-10-04 19:44:00+00:00,10147.0,1542.0,0.0,0.0,0.0,632.0
497,391,2023-10-04 19:44:05+00:00,7912.0,1163.0,0.0,0.0,0.0,519.0
498,391,2023-10-04 19:44:10+00:00,13953.0,2109.0,0.0,0.0,0.0,765.0


In [12]:
#Import partial tables and append -> for propagation time

publishMessage = pd.DataFrame()
deliverMessage = pd.DataFrame()

for index, row in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/mess2_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_mess = pd.DataFrame(data)

    data = pd.read_csv('../../datasets/partialTables/pub_'+str(row['experiment'])+'.csv', header=0,  index_col=0)
    query_pub = pd.DataFrame(data)
    
    deliverMessage = pd.concat([query_mess, deliverMessage])
    publishMessage = pd.concat([publishMessage, query_pub])

    del query_pub
    del query_mess

    gc.collect()

deliverMessage.head(10)


,_time,messageID,experiment
0,2024-01-20 06:23:42.139365+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
1,2024-01-20 06:23:42.139457+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
2,2024-01-20 06:23:42.139631+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
3,2024-01-20 06:23:42.140159+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
4,2024-01-20 06:23:42.140332+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
5,2024-01-20 06:23:42.140430+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
6,2024-01-20 06:23:42.140436+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
7,2024-01-20 06:23:42.141264+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
8,2024-01-20 06:23:42.141383+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762
9,2024-01-20 06:23:42.141725+00:00,ACQIARIgI1fcHl95jQzZMa2FaRW3bAKVmBA6C+PstfIuT4...,762


In [13]:
publishMessage['_time'] = pd.to_datetime(publishMessage["_time"], format='mixed')#.tz_localize(None)
deliverMessage['_time'] = pd.to_datetime(deliverMessage["_time"], format='mixed')#.tz_localize(None)

# def calcAverageTime(publish, received, expTime, parameter):
expTime = experiments
publish = publishMessage
received = deliverMessage

publish = publish[['_time', 'messageID','experiment']]
received = received[['_time', 'messageID', 'experiment']]

joined = publish.merge(received, on=['messageID', 'experiment'])
joined['diff'] = ((joined['_time_y'] - joined['_time_x'])/ pd.Timedelta(microseconds=1)).astype(int)
joined = joined.loc[joined["diff"] >= 0].dropna()
joined.head(10)


,_time_x,messageID,experiment,_time_y,diff
0,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134903+00:00,753
1,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136298+00:00,2148
2,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.136886+00:00,2736
3,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137090+00:00,2940
4,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137205+00:00,3055
5,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137276+00:00,3126
6,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137353+00:00,3203
7,2023-10-04 18:52:04.134150+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.137377+00:00,3227
8,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.134975+00:00,308
9,2023-10-04 18:52:04.134667+00:00,ACQIARIgD0L9DFUrEMrmskLqnXInFpXWSpAe/CjcDO6NgA...,390,2023-10-04 18:52:04.135169+00:00,502


In [14]:
df = joined.drop(columns=['_time_x', 'messageID']).rename(columns={'_time_y' : '_time'})

df.reset_index(inplace=True)
df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)
by_time = df.groupby([df['experiment'],pd.Grouper(key="_time", freq='5s')])['diff'].agg('mean').reset_index()
by_time.head(100)

/tmp/ipykernel_3299/2407133802.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['_time'] = pd.to_datetime(df["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,diff
0,390,2023-10-04 18:52:00+00:00,1856.745902
1,390,2023-10-04 18:52:05+00:00,1722.702500
2,390,2023-10-04 18:52:10+00:00,1779.995058
3,390,2023-10-04 18:52:15+00:00,2357.267894
4,390,2023-10-04 18:52:20+00:00,1779.471396
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,1510.008584
96,390,2023-10-04 19:00:00+00:00,1567.277027
97,390,2023-10-04 19:00:05+00:00,1740.478261
98,390,2023-10-04 19:00:10+00:00,1647.693966


In [15]:
prop = by_time.rename(columns={'diff':'propagationTime'})

prop = prop.merge(eventsTS, on=['experiment', '_time'], how='outer')
prop.head(100)

,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave
0,390,2023-10-04 18:52:00+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0
1,390,2023-10-04 18:52:05+00:00,1722.702500,4936.0,673.0,0.0,0.0,0.0,698.0
2,390,2023-10-04 18:52:10+00:00,1779.995058,8457.0,1156.0,0.0,0.0,0.0,689.0
3,390,2023-10-04 18:52:15+00:00,2357.267894,7681.0,1032.0,0.0,0.0,0.0,343.0
4,390,2023-10-04 18:52:20+00:00,1779.471396,5874.0,796.0,0.0,0.0,0.0,695.0
...,...,...,...,...,...,...,...,...,...
95,390,2023-10-04 18:59:55+00:00,1510.008584,4378.0,654.0,0.0,0.0,0.0,393.0
96,390,2023-10-04 19:00:00+00:00,1567.277027,2748.0,408.0,0.0,0.0,0.0,195.0
97,390,2023-10-04 19:00:05+00:00,1740.478261,4453.0,669.0,0.0,0.0,0.0,385.0
98,390,2023-10-04 19:00:10+00:00,1647.693966,4292.0,650.0,0.0,0.0,0.0,357.0


In [16]:
prop.to_csv('./propagationTime5s.csv')

In [17]:
#Bandwidth

#Load rpc dataset
# data = pd.read_csv('./Extraction/rpc.csv',header=0, index_col=0)
# rpc = pd.DataFrame(data)
# rpc = rpc.rename(columns={'rpc':'count'})

# rpc.head(10)

rpc = pd.DataFrame()


for idx, exec in experiments.iterrows():
    data = pd.read_csv('../../datasets/partialTables/rpc_'+str(exec['experiment'])+'.csv', header=0,  index_col=0)
    query_rpc = pd.DataFrame(data)
        
    rpc = pd.concat([rpc,query_rpc])

    del query_rpc
    gc.collect()
        

rpc.head(10)

,_time,_measurement,experiment,start,end
0,2023-10-04 18:52:04.134925+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
1,2023-10-04 18:52:04.135293+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
2,2023-10-04 18:52:04.135840+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
3,2023-10-04 18:52:04.135852+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
4,2023-10-04 18:52:04.135928+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
5,2023-10-04 18:52:04.135930+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
6,2023-10-04 18:52:04.135938+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
7,2023-10-04 18:52:04.135962+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
8,2023-10-04 18:52:04.135988+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518
9,2023-10-04 18:52:04.136030+00:00,recvRPC,390,2023-10-04 18:52:03.0807730,2023-10-04 19:22:03.0866518


In [18]:
rpc = rpc.drop(columns=['start', 'end'])
                       
rpc.reset_index(inplace=True)
rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)
by_time = rpc.groupby([rpc['experiment'],pd.Grouper(key="_time", freq='5s')])['index'].agg('count').reset_index()
# by_time.head(100)

rpc=by_time.rename(columns={'index':'rpc'})

/tmp/ipykernel_3299/1580870231.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  rpc['_time'] = pd.to_datetime(rpc["_time"], format='mixed', infer_datetime_format=True)


In [19]:
rpc.head(10)

,experiment,_time,rpc
0,390,2023-10-04 18:52:00+00:00,4243
1,390,2023-10-04 18:52:05+00:00,5906
2,390,2023-10-04 18:52:10+00:00,9869
3,390,2023-10-04 18:52:15+00:00,8850
4,390,2023-10-04 18:52:20+00:00,6954
5,390,2023-10-04 18:52:25+00:00,9155
6,390,2023-10-04 18:52:30+00:00,7734
7,390,2023-10-04 18:52:35+00:00,8255
8,390,2023-10-04 18:52:40+00:00,9376
9,390,2023-10-04 18:52:45+00:00,6919


In [20]:
#Total bandwidth

# data = pd.read_csv('../datasets/timeSeries1s.csv', header=0,  index_col=0)
# message = pd.DataFrame(data)

# idx = [3.0]
# message = message[message["type"].isin(idx)]
# message = message.drop(columns=["type", "identifier"])

message=received

message.reset_index(inplace=True)
message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)
by_time = message.groupby([message['experiment'],pd.Grouper(key="_time", freq='5s')])['messageID'].agg('count').reset_index()
message = by_time

message.head(10)

/tmp/ipykernel_3299/3808709829.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  message['_time'] = pd.to_datetime(message["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,messageID
0,390,2023-10-04 18:52:00+00:00,521
1,390,2023-10-04 18:52:05+00:00,678
2,390,2023-10-04 18:52:10+00:00,1158
3,390,2023-10-04 18:52:15+00:00,1042
4,390,2023-10-04 18:52:20+00:00,802
5,390,2023-10-04 18:52:25+00:00,1080
6,390,2023-10-04 18:52:30+00:00,932
7,390,2023-10-04 18:52:35+00:00,977
8,390,2023-10-04 18:52:40+00:00,1107
9,390,2023-10-04 18:52:45+00:00,829


In [21]:
message = message.rename(columns={'messageID':'count'})
rpc = rpc.rename(columns={'rpc': 'count'})
total_bandwidth = pd.concat([rpc, message])

total_bandwidth.reset_index(inplace=True)
total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)
by_time = total_bandwidth.groupby([total_bandwidth['experiment'],pd.Grouper(key="_time", freq='5s')])['count'].agg('sum').reset_index()
by_time.head(100)

/tmp/ipykernel_3299/448925867.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  total_bandwidth['_time'] = pd.to_datetime(total_bandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,count
0,390,2023-10-04 18:52:00+00:00,4764
1,390,2023-10-04 18:52:05+00:00,6584
2,390,2023-10-04 18:52:10+00:00,11027
3,390,2023-10-04 18:52:15+00:00,9892
4,390,2023-10-04 18:52:20+00:00,7756
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,5843
96,390,2023-10-04 19:00:00+00:00,3635
97,390,2023-10-04 19:00:05+00:00,5929
98,390,2023-10-04 19:00:10+00:00,5724


In [22]:
by_time['totalBandwidth'] = by_time['count']

totalBandwidth = by_time.drop(columns=['count'])
totalBandwidth.head(10)

,experiment,_time,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,4764
1,390,2023-10-04 18:52:05+00:00,6584
2,390,2023-10-04 18:52:10+00:00,11027
3,390,2023-10-04 18:52:15+00:00,9892
4,390,2023-10-04 18:52:20+00:00,7756
5,390,2023-10-04 18:52:25+00:00,10235
6,390,2023-10-04 18:52:30+00:00,8666
7,390,2023-10-04 18:52:35+00:00,9232
8,390,2023-10-04 18:52:40+00:00,10483
9,390,2023-10-04 18:52:45+00:00,7748


In [23]:
#gossip bandwidth
gossipBandwidth = rpc
gossipBandwidth['gossipBandwidth'] = gossipBandwidth['count']/5
gossipBandwidth = gossipBandwidth.drop(columns=['count'])

gossipBandwidth.head(100)

,experiment,_time,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,848.6
1,390,2023-10-04 18:52:05+00:00,1181.2
2,390,2023-10-04 18:52:10+00:00,1973.8
3,390,2023-10-04 18:52:15+00:00,1770.0
4,390,2023-10-04 18:52:20+00:00,1390.8
...,...,...,...
95,390,2023-10-04 18:59:55+00:00,1037.4
96,390,2023-10-04 19:00:00+00:00,645.4
97,390,2023-10-04 19:00:05+00:00,1051.8
98,390,2023-10-04 19:00:10+00:00,1014.8


In [24]:
#MessageBandwidth

messageBandwidth = message
messageBandwidth['messageBandwidth'] = messageBandwidth['count']/5
messageBandwidth = messageBandwidth.drop(columns=['count'])

messageBandwidth.head(10)

,experiment,_time,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,104.2
1,390,2023-10-04 18:52:05+00:00,135.6
2,390,2023-10-04 18:52:10+00:00,231.6
3,390,2023-10-04 18:52:15+00:00,208.4
4,390,2023-10-04 18:52:20+00:00,160.4
5,390,2023-10-04 18:52:25+00:00,216.0
6,390,2023-10-04 18:52:30+00:00,186.4
7,390,2023-10-04 18:52:35+00:00,195.4
8,390,2023-10-04 18:52:40+00:00,221.4
9,390,2023-10-04 18:52:45+00:00,165.8


In [25]:
prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)

messBand = prop.merge(messageBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

messBand.head(500)

/tmp/ipykernel_3299/2873079763.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  prop['_time'] = pd.to_datetime(prop["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_3299/2873079763.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messageBandwidth['_time'] = pd.to_datetime(messageBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth
0,390,2023-10-04 18:52:00+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,104.2
1,390,2023-10-04 18:52:05+00:00,1722.702500,4936.0,673.0,0.0,0.0,0.0,698.0,135.6
2,390,2023-10-04 18:52:10+00:00,1779.995058,8457.0,1156.0,0.0,0.0,0.0,689.0,231.6
3,390,2023-10-04 18:52:15+00:00,2357.267894,7681.0,1032.0,0.0,0.0,0.0,343.0,208.4
4,390,2023-10-04 18:52:20+00:00,1779.471396,5874.0,796.0,0.0,0.0,0.0,695.0,160.4
...,...,...,...,...,...,...,...,...,...,...
495,391,2023-10-04 19:44:00+00:00,1757.885167,10147.0,1542.0,0.0,0.0,0.0,632.0,309.4
496,391,2023-10-04 19:44:05+00:00,1614.945887,7912.0,1163.0,0.0,0.0,0.0,519.0,233.2
497,391,2023-10-04 19:44:10+00:00,1681.838275,13953.0,2109.0,0.0,0.0,0.0,765.0,423.4
498,391,2023-10-04 19:44:15+00:00,1846.494196,8450.0,1274.0,0.0,0.0,0.0,635.0,256.0


In [26]:
messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

totalBand = messBand.merge(totalBandwidth, on=['experiment', '_time'], how='outer').fillna(0)
totalBand['totalBandwidth'] = (totalBand['totalBandwidth']+totalBand['messageDuplicated'])/5

totalBand.head(500)

/tmp/ipykernel_3299/511213079.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  messBand['_time'] = pd.to_datetime(messBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_3299/511213079.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth
0,390,2023-10-04 18:52:00+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,104.2,1702.4
1,390,2023-10-04 18:52:05+00:00,1722.702500,4936.0,673.0,0.0,0.0,0.0,698.0,135.6,2304.0
2,390,2023-10-04 18:52:10+00:00,1779.995058,8457.0,1156.0,0.0,0.0,0.0,689.0,231.6,3896.8
3,390,2023-10-04 18:52:15+00:00,2357.267894,7681.0,1032.0,0.0,0.0,0.0,343.0,208.4,3514.6
4,390,2023-10-04 18:52:20+00:00,1779.471396,5874.0,796.0,0.0,0.0,0.0,695.0,160.4,2726.0
...,...,...,...,...,...,...,...,...,...,...,...
495,391,2023-10-04 19:44:00+00:00,1757.885167,10147.0,1542.0,0.0,0.0,0.0,632.0,309.4,4726.2
496,391,2023-10-04 19:44:05+00:00,1614.945887,7912.0,1163.0,0.0,0.0,0.0,519.0,233.2,3669.0
497,391,2023-10-04 19:44:10+00:00,1681.838275,13953.0,2109.0,0.0,0.0,0.0,765.0,423.4,6475.0
498,391,2023-10-04 19:44:15+00:00,1846.494196,8450.0,1274.0,0.0,0.0,0.0,635.0,256.0,3942.8


In [27]:
totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)

gossipBand = totalBand.merge(gossipBandwidth, on=['experiment', '_time'], how='outer').fillna(0)

gossipBand.head(500)

/tmp/ipykernel_3299/187803194.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  totalBand['_time'] = pd.to_datetime(totalBand["_time"], format='mixed', infer_datetime_format=True)
/tmp/ipykernel_3299/187803194.py:2: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  gossipBandwidth['_time'] = pd.to_datetime(totalBandwidth["_time"], format='mixed', infer_datetime_format=True)


,experiment,_time,propagationTime,messageDuplicated,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,104.2,1702.4,848.6
1,390,2023-10-04 18:52:05+00:00,1722.702500,4936.0,673.0,0.0,0.0,0.0,698.0,135.6,2304.0,1181.2
2,390,2023-10-04 18:52:10+00:00,1779.995058,8457.0,1156.0,0.0,0.0,0.0,689.0,231.6,3896.8,1973.8
3,390,2023-10-04 18:52:15+00:00,2357.267894,7681.0,1032.0,0.0,0.0,0.0,343.0,208.4,3514.6,1770.0
4,390,2023-10-04 18:52:20+00:00,1779.471396,5874.0,796.0,0.0,0.0,0.0,695.0,160.4,2726.0,1390.8
...,...,...,...,...,...,...,...,...,...,...,...,...
495,391,2023-10-04 19:44:00+00:00,1757.885167,10147.0,1542.0,0.0,0.0,0.0,632.0,309.4,4726.2,2387.4
496,391,2023-10-04 19:44:05+00:00,1614.945887,7912.0,1163.0,0.0,0.0,0.0,519.0,233.2,3669.0,1853.4
497,391,2023-10-04 19:44:10+00:00,1681.838275,13953.0,2109.0,0.0,0.0,0.0,765.0,423.4,6475.0,3261.0
498,391,2023-10-04 19:44:15+00:00,1846.494196,8450.0,1274.0,0.0,0.0,0.0,635.0,256.0,3942.8,1996.8


In [28]:
consDiscovery = gossipBand.rename(columns={'messageDuplicated' : 'messageOverhead'})#.drop(columns=['bandwidth'])
consDiscovery.to_csv('./consDiscovery5s.csv')

consDiscovery.head(10)

,experiment,_time,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth
0,390,2023-10-04 18:52:00+00:00,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,104.2,1702.4,848.6
1,390,2023-10-04 18:52:05+00:00,1722.702500,4936.0,673.0,0.0,0.0,0.0,698.0,135.6,2304.0,1181.2
2,390,2023-10-04 18:52:10+00:00,1779.995058,8457.0,1156.0,0.0,0.0,0.0,689.0,231.6,3896.8,1973.8
3,390,2023-10-04 18:52:15+00:00,2357.267894,7681.0,1032.0,0.0,0.0,0.0,343.0,208.4,3514.6,1770.0
4,390,2023-10-04 18:52:20+00:00,1779.471396,5874.0,796.0,0.0,0.0,0.0,695.0,160.4,2726.0,1390.8
5,390,2023-10-04 18:52:25+00:00,1763.668550,7815.0,1078.0,0.0,0.0,0.0,644.0,216.0,3610.0,1831.0
6,390,2023-10-04 18:52:30+00:00,1876.800830,6706.0,928.0,0.0,0.0,0.0,324.0,186.4,3074.4,1546.8
7,390,2023-10-04 18:52:35+00:00,1663.212644,7044.0,970.0,0.0,0.0,0.0,639.0,195.4,3255.2,1651.0
8,390,2023-10-04 18:52:40+00:00,1953.664855,8019.0,1105.0,0.0,0.0,2.0,648.0,221.4,3700.4,1875.2
9,390,2023-10-04 18:52:45+00:00,1967.343035,5974.0,825.0,0.0,0.0,0.0,320.0,165.8,2744.4,1383.8


In [29]:
consFinal = consDiscovery.merge(experiments, on='experiment')
consFinal = consFinal.drop(columns=['_time', 'start', 'end', 'startUnix', 'endUnix', 'parameter'])#, 'originalStart', 'originalEnd']) 

consFinal.to_csv('../consolidated5s.csv')
consFinal.head(10)


,experiment,propagationTime,messageOverhead,messageReceived,graft,prune,iwant,ihave,messageBandwidth,totalBandwidth,gossipBandwidth,topology,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval
0,390,1856.745902,3748.0,520.0,0.0,0.0,0.0,0.0,104.2,1702.4,848.6,unl,8,6,12,4,8,2,0.25,1.0
1,390,1722.702500,4936.0,673.0,0.0,0.0,0.0,698.0,135.6,2304.0,1181.2,unl,8,6,12,4,8,2,0.25,1.0
2,390,1779.995058,8457.0,1156.0,0.0,0.0,0.0,689.0,231.6,3896.8,1973.8,unl,8,6,12,4,8,2,0.25,1.0
3,390,2357.267894,7681.0,1032.0,0.0,0.0,0.0,343.0,208.4,3514.6,1770.0,unl,8,6,12,4,8,2,0.25,1.0
4,390,1779.471396,5874.0,796.0,0.0,0.0,0.0,695.0,160.4,2726.0,1390.8,unl,8,6,12,4,8,2,0.25,1.0
5,390,1763.668550,7815.0,1078.0,0.0,0.0,0.0,644.0,216.0,3610.0,1831.0,unl,8,6,12,4,8,2,0.25,1.0
6,390,1876.800830,6706.0,928.0,0.0,0.0,0.0,324.0,186.4,3074.4,1546.8,unl,8,6,12,4,8,2,0.25,1.0
7,390,1663.212644,7044.0,970.0,0.0,0.0,0.0,639.0,195.4,3255.2,1651.0,unl,8,6,12,4,8,2,0.25,1.0
8,390,1953.664855,8019.0,1105.0,0.0,0.0,2.0,648.0,221.4,3700.4,1875.2,unl,8,6,12,4,8,2,0.25,1.0
9,390,1967.343035,5974.0,825.0,0.0,0.0,0.0,320.0,165.8,2744.4,1383.8,unl,8,6,12,4,8,2,0.25,1.0
